In [ ]:
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
from vf_portalytics.model import PredictionModel
from vf_portalytics.dataset import DataSet
from vf_portalytics.tool import mean_absolute_percentage_error, forecast_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils.validation import check_array
from sklearn import linear_model, ensemble, svm

In [ ]:
# we load data and create the basics
dataset = DataSet('example', path='example_dir')
df = dataset.data_df
prediction_model = PredictionModel('example', path='example_dir')